# 使用FireworksAI、LangChain和MongoDB进行Agent开发

本教程展示了如何使用FireworksAI、LangChain和MongoDB来构建一个智能Agent。我们将使用FireworksAI的API来运行大语言模型,使用LangChain来构建Prompt和工具链,使用MongoDB来存储和检索数据。

要运行这个notebook,你需要:
1. 一个FireworksAI API密钥 - 在[这里注册](https://app.fireworks.ai)
2. 一个MongoDB实例

In [ ]:
import os
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()

# 设置环境变量
os.environ["FIREWORKS_API_KEY"] = "<你的FIREWORKS_API_KEY>"

## 配置与依赖项

运行以下命令来安装所需的依赖包:

In [ ]:
# !pip install fireworks-ai langchain pymongo
# !pip install 'requests>=2.31.0'  # fireworks-ai需要更新的requests

In [ ]:
import json
import logging
from datetime import datetime
from typing import Any, Dict, List

from fireworks.client import Fireworks
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_fireworks import ChatFireworks
from pymongo import MongoClient

# MongoDB连接字符串
MONGO_CONNECTION_STRING = "<你的MONGO_CONNECTION_STRING>"

# 设置日志级别
logging.basicConfig(level=logging.INFO)

## MongoDB设置

我们将创建一个MongoDB集合来存储对话历史。每个文档将包含以下字段:
- message_id: 消息的唯一标识符
- timestamp: 消息创建时间
- role: 发送者的角色(用户或助手)
- content: 消息内容

In [ ]:
# 连接到MongoDB
client = MongoClient(MONGO_CONNECTION_STRING)
db = client["chatbot"]
collection = db["conversations"]

def store_message(role: str, content: str) -> str:
    """在MongoDB中存储消息。

    Args:
        role: 消息发送者的角色(用户或助手)
        content: 消息内容

    Returns:
        str: 已存储消息的ID
    """
    document = {
        "timestamp": datetime.utcnow(),
        "role": role,
        "content": content
    }
    result = collection.insert_one(document)
    return str(result.inserted_id)

def get_conversation_history(limit: int = 10) -> List[Dict[str, Any]]:
    """从MongoDB检索最近的对话历史。

    Args:
        limit: 要检索的消息数量

    Returns:
        List[Dict[str, Any]]: 对话历史列表
    """
    return list(
        collection.find(
            {},
            {"_id": 0, "timestamp": 0}
        )
        .sort("timestamp", -1)
        .limit(limit)
    )

/Users/richmondalake/miniconda3/envs/langchain_workarea/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Repo card metadata block was not found. Setting CardData to empty.
Generating train split: 50000 examples [00:01, 38699.64 examples/s]


## FireworksAI和LangChain设置

现在我们将设置FireworksAI语言模型和LangChain组件。

In [ ]:
# 初始化FireworksAI聊天模型
llm = ChatFireworks(
    model="accounts/fireworks/models/llama-v2-7b-chat",
    max_tokens=2048,
    temperature=0.7,
)

## 定义提示模板

我们将创建一个提示模板,它会:
1. 包含对话历史记录
2. 请求助手以友好和专业的方式回应

In [ ]:
# 定义提示模板
template = """你是一个友好和专业的AI助手。

对话历史记录:
{conversation_history}

用户: {user_input}

请以专业和有帮助的方式回应。
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["conversation_history", "user_input"]
)

# 创建LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

## 主对话循环

让我们创建一个函数来处理对话循环:

In [ ]:
def chat_loop():
    """运行主对话循环。

    处理用户输入、生成响应、存储消息并显示响应。
    """
    print("开始对话! 输入'退出'来结束对话。\n")
    
    while True:
        # 获取用户输入
        user_input = input("你: ")
        if user_input.lower() == "退出":
            break

        # 存储用户消息
        store_message("user", user_input)

        # 获取对话历史
        history = get_conversation_history()
        history_str = "\n".join(
            [f"{msg['role']}: {msg['content']}" for msg in reversed(history)]
        )

        # 生成响应
        response = chain.run(
            conversation_history=history_str,
            user_input=user_input
        )

        # 存储助手响应
        store_message("assistant", response)

        # 显示响应
        print(f"\n助手: {response}\n")

## 运行对话

现在我们可以运行对话循环了:

In [ ]:
if __name__ == "__main__":
    chat_loop()

这就完成了一个基本的对话Agent! 它具有以下功能:

1. 使用FireworksAI的语言模型生成响应
2. 使用LangChain处理提示模板和链式操作
3. 使用MongoDB存储和检索对话历史记录

你可以通过以下方式扩展这个Agent:

1. 添加更多工具和功能
2. 改进提示工程
3. 实现更复杂的对话管理
4. 添加错误处理和重试逻辑
5. 添加用户认证和授权
6. 实现对话上下文管理

## 进一步阅读

- [FireworksAI文档](https://fireworks.ai/docs)
- [LangChain文档](https://python.langchain.com/docs/get_started/introduction.html)
- [MongoDB文档](https://www.mongodb.com/docs/)